##Installing the required packages

In [ ]:
!pip install pycparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install anytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 545 kB/s 


In [ ]:
!pip show pycparser

Name: pycparser
Version: 2.21
Summary: C parser in Python
Home-page: https://github.com/eliben/pycparser
Author: Eli Bendersky
Author-email: eliben@gmail.com
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: cffi


In [ ]:
#import os
#os.listdir('/usr/local/lib/python3.7/dist-packages/pycparser')

['c_parser.py',
 '__pycache__',
 'plyparser.py',
 'ast_transforms.py',
 'lextab.py',
 '_c_ast.cfg',
 '__init__.py',
 'c_ast.py',
 '_ast_gen.py',
 'utils',
 'ply',
 '_build_tables.py',
 'c_generator.py',
 'c_lexer.py',
 'yacctab.py']

In [ ]:
'''
#to remove comments 
import re
def comment_remover(text):
    def replacer(match):
        s = match.group(0)
        if s.startswith('/'):
            return ""
        else:
            return s
    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )
    return re.sub(pattern, replacer, text)
'''

In [ ]:
import pycparser
filename = '/content/testPP.c'
ast = pycparser.parse_file(filename, use_cpp=True)        
ast.show()

FileAST: 
  Typedef: size_t, [], ['typedef']
    TypeDecl: size_t, [], None
      IdentifierType: ['int']
  Typedef: __builtin_va_list, [], ['typedef']
    TypeDecl: __builtin_va_list, [], None
      IdentifierType: ['int']
  Typedef: __gnuc_va_list, [], ['typedef']
    TypeDecl: __gnuc_va_list, [], None
      IdentifierType: ['int']
  Typedef: va_list, [], ['typedef']
    TypeDecl: va_list, [], None
      IdentifierType: ['int']
  Typedef: __int8_t, [], ['typedef']
    TypeDecl: __int8_t, [], None
      IdentifierType: ['int']
  Typedef: __uint8_t, [], ['typedef']
    TypeDecl: __uint8_t, [], None
      IdentifierType: ['int']
  Typedef: __int16_t, [], ['typedef']
    TypeDecl: __int16_t, [], None
      IdentifierType: ['int']
  Typedef: __uint16_t, [], ['typedef']
    TypeDecl: __uint16_t, [], None
      IdentifierType: ['int']
  Typedef: __int_least16_t, [], ['typedef']
    TypeDecl: __int_least16_t, [], None
      IdentifierType: ['int']
  Typedef: __uint_least16_t, [], ['typedef']

In [ ]:
'''
from __future__ import print_function
import sys

# This is not required if you've installed pycparser into
# your site-packages/ with setup.py
#
sys.path.extend(['.', '..'])

from pycparser import c_parser


text = r"""
    typedef int Node, Hash;
    void HashPrint(Hash* hash, void (*PrintFunc)(char*, char*))
    {
        unsigned int i;
        if (hash == NULL || hash->heads == NULL)
            return;
        for (i = 0; i < hash->table_size; ++i)
        {
            Node* temp = hash->heads[i];
            while (temp != NULL)
            {
                PrintFunc(temp->entry->key, temp->entry->value);
                temp = temp->next;
            }
        }
    }
"""

parser = c_parser.CParser()
ast = parser.parse(text, filename='<none>')

# Uncomment the following line to see the AST in a nice, human
# readable way. show() is the most useful tool in exploring ASTs
# created by pycparser. See the c_ast.py file for the options you
# can pass it.

#ast.show(showcoord=True)

# OK, we've seen that the top node is FileAST. This is always the
# top node of the AST. Its children are "external declarations",
# and are stored in a list called ext[] (see _c_ast.cfg for the
# names and types of Nodes and their children).
# As you see from the printout, our AST has two Typedef children
# and one FuncDef child.
# Let's explore FuncDef more closely. As I've mentioned, the list
# ext[] holds the children of FileAST. Since the function
# definition is the third child, it's ext[2]. Uncomment the
# following line to show it:
#print("--------------------------------------------------------")
#count=0
#for i in ast.ext:
  #count+=1
  #print(count,":---------------------------------------------------------")
  #print(i)

# A FuncDef consists of a declaration, a list of parameter
# declarations (for K&R style function definitions), and a body.
# First, let's examine the declaration.
func = ast.ext[2]
func.show()
function_decl = ast.ext[2].decl

# function_decl, like any other declaration, is a Decl. Its type child
# i

#function_decl.type.show()
#function_decl.type.args.show()

# The following displays the name and type of each argument:

#for param_decl in function_decl.type.args.params:
    #print('Arg name: %s' % param_decl.name)
    #print('Type:')
    #param_decl.type.show(offset=6)

# The body is of FuncDef is a Compound, which is a placeholder for a block
# surrounded by {} (You should be reading _c_ast.cfg parallel to this
# explanations a FuncDecl, which has a return type and arguments stored in a
# ParamList node and seeing these things with your own eyes).
# Let's see the block's declarations:

function_body = ast.ext[2].body

# The following displays the declarations and statements in the function
# body

#for decl in function_body.block_items:
    #decl.show()

# We can see a single variable declaration, i, declared to be a simple type
# declaration of type 'unsigned int', followed by statements.

# block_items is a list, so the third element is the For statement:

for_stmt = function_body.block_items[2]
#for_stmt.show()

# As you can see in _c_ast.cfg, For's children are 'init, cond,
# next' for the respective parts of the 'for' loop specifier,
# and stmt, which is either a single stmt or a Compound if there's
# a block.
#
# Let's dig deeper, to the while statement inside the for loop:

while_stmt = for_stmt.stmt.block_items[1]
#while_stmt.show()

# While is simpler, it only has a condition node and a stmt node.
# The condition:

while_cond = while_stmt.cond
#while_cond.show()

# Note that it's a BinaryOp node - the basic constituent of
# expressions in our AST. BinaryOp is the expression tree, with
# left and right nodes as children. It also has the op attribute,
# which is just the string representation of the operator.

#print(while_cond.op)
#while_cond.left.show()
#while_cond.right.show()


# That's it for the example. I hope you now see how easy it is to explore the
# AST created by pycparser. Although on the surface it is quite complex and has
# a lot of node types, this is the inherent complexity of the C language every
# parser/compiler designer has to cope with.
# Using the tools provided by the c_ast package it's easy to explore the
# structure of AST nodes and write code that processes them.
# Specifically, see the cdecl.py example for a non-trivial demonstration of what
# you can do by recursively going through the AST.
'''

FuncDef: 
  Decl: HashPrint, [], [], [], []
    FuncDecl: 
      ParamList: 
        Decl: hash, [], [], [], []
          PtrDecl: []
            TypeDecl: hash, [], None
              IdentifierType: ['Hash']
        Decl: PrintFunc, [], [], [], []
          PtrDecl: []
            FuncDecl: 
              ParamList: 
                Typename: None, [], None
                  PtrDecl: []
                    TypeDecl: None, [], None
                      IdentifierType: ['char']
                Typename: None, [], None
                  PtrDecl: []
                    TypeDecl: None, [], None
                      IdentifierType: ['char']
              TypeDecl: PrintFunc, [], None
                IdentifierType: ['void']
      TypeDecl: HashPrint, [], None
        IdentifierType: ['void']
  Compound: 
    Decl: i, [], [], [], []
      TypeDecl: i, [], None
        IdentifierType: ['unsigned', 'int']
    If: 
      BinaryOp: ||
        BinaryOp: ==
          ID: hash
          ID: NULL

In [ ]:
'''
for i in ast.ext[2]:
  print("--------------")
  print(i.children())
'''

--------------
(('type', FuncDecl(args=ParamList(params=[Decl(name='hash',
                                     quals=[
                                           ],
                                     align=[
                                           ],
                                     storage=[
                                             ],
                                     funcspec=[
                                              ],
                                     type=PtrDecl(quals=[
                                                        ],
                                                  type=TypeDecl(declname='hash',
                                                                quals=[
                                                                      ],
                                                                align=None,
                                                                type=IdentifierType(names=['Hash'
                                  

##Generating AST for the code

In [ ]:
import pycparser
from pycparser import c_parser

def program_parser(filename):
    #import pycparser
    ast = pycparser.parse_file(filename, use_cpp=True)  
    #parser = c_parser.CParser()
    #ast = parser.parse(func, filename='<none>')
    #print(ast)
    return ast

In [ ]:
filename = '/content/testPP.c'
ast = program_parser(filename)

##Getting the tokens from the nodes

In [ ]:
import pycparser
def get_token(node):
    token = ''
    if isinstance(node, pycparser.c_ast.FileAST):
        token = node.__class__.__name__
        return token
    elif isinstance(node[1], str):
        token = node[1]
        return token
        #print(token)
    elif isinstance(node[1], pycparser.c_ast.Node):
        token = node[1].__class__.__name__
    #print(token)
    
    if len(get_children(node))==0:
        attr_names = node[1].attr_names
        if attr_names:
            if 'names' in attr_names:
                token = node[1].names[0]
            elif 'name' in attr_names:
                token = node[1].name
            else:
                token = node[1].value
    else:
        if token == 'TypeDecl':
            token = node[1].declname
        if node[1].attr_names:
            attr_names = node[1].attr_names
            if 'op' in attr_names:
                if node[1].op[0] == 'p':
                    token = node[1].op[1:]
                else:
                    token = node[1].op
 
    if token == '':
        token = node[1].__class__.__name__
    return token

##Getting the children nodes and getting a tree 

In [ ]:
from anytree import Node

def get_children(root):
    if isinstance(root, pycparser.c_ast.FileAST):
        children = root.children()
    elif isinstance(root[1], pycparser.c_ast.Node):
        children = root[1].children()
    elif isinstance(root, set):
        children = list(root)
    else:
        children = []

    def expand(nested_list):
        for item in nested_list:
            if isinstance(item, list):
                for sub_item in expand(item):
                    yield sub_item
            elif item:
                yield item

    return list(expand(children))

def get_trees(current_node, parent_node, order):
    token, children = get_token(current_node), get_children(current_node)
    node = Node([order,token], parent=parent_node, order=order)
    #print(parent_node)
    #if(parent_node.name=='')
    p = str(parent_node.name[0])+"--"+str(parent_node.name[1])
    c = str(order)+"--"+str(token)
    nodes.append((p,c))
    #print(nodes)
    for child_order in range(len(children)):
        get_trees(children[child_order], node, order+str(int(child_order)+1))

In [ ]:
c_code = ast

In [ ]:
from anytree import Node
# Initialize head node of the code.
head = Node(["1",get_token(c_code)])
# Recursively construct AST tree.
nodes = []
nodes.append((None,str(head.name[1])+str(head.name[0])))
for child_order in range(len(get_children(c_code))):
    get_trees(get_children(c_code)[child_order], head, "1"+str(int(child_order)+1))

In [ ]:
for i in nodes:
  print(i)

(None, 'FileAST1')
('1FileAST', '11Typedef')
('11Typedef', '111size_t')
('111size_t', '1111int')
('1FileAST', '12Typedef')
('12Typedef', '121__builtin_va_list')
('121__builtin_va_list', '1211int')
('1FileAST', '13Typedef')
('13Typedef', '131__gnuc_va_list')
('131__gnuc_va_list', '1311int')
('1FileAST', '14Typedef')
('14Typedef', '141va_list')
('141va_list', '1411int')
('1FileAST', '15Typedef')
('15Typedef', '151__int8_t')
('151__int8_t', '1511int')
('1FileAST', '16Typedef')
('16Typedef', '161__uint8_t')
('161__uint8_t', '1611int')
('1FileAST', '17Typedef')
('17Typedef', '171__int16_t')
('171__int16_t', '1711int')
('1FileAST', '18Typedef')
('18Typedef', '181__uint16_t')
('181__uint16_t', '1811int')
('1FileAST', '19Typedef')
('19Typedef', '191__int_least16_t')
('191__int_least16_t', '1911int')
('1FileAST', '110Typedef')
('110Typedef', '1101__uint_least16_t')
('1101__uint_least16_t', '11011int')
('1FileAST', '111Typedef')
('111Typedef', '1111__int32_t')
('1111__int32_t', '11111int')
('1Fi

In [ ]:
#from anytree import RenderTree
#print(RenderTree(head))

In [ ]:
from pprint import pprint  # just for nice printing
#from anytree import AnyNode
from anytree.exporter import DictExporter
exporter = DictExporter()
pprint(exporter.export(head))

{'children': [{'children': [{'children': [{'name': ['1111', 'int'],
                                           'order': '1111'}],
                             'name': ['111', 'size_t'],
                             'order': '111'}],
               'name': ['11', 'Typedef'],
               'order': '11'},
              {'children': [{'children': [{'name': ['1211', 'int'],
                                           'order': '1211'}],
                             'name': ['121', '__builtin_va_list'],
                             'order': '121'}],
               'name': ['12', 'Typedef'],
               'order': '12'},
              {'children': [{'children': [{'name': ['1311', 'int'],
                                           'order': '1311'}],
                             'name': ['131', '__gnuc_va_list'],
                             'order': '131'}],
               'name': ['13', 'Typedef'],
               'order': '13'},
              {'children': [{'children': [{'name': ['1411', 'in

In [ ]:
'''
#from _typeshed import OpenBinaryModeUpdating
from pycparser import c_ast, parse_file
import pycparser
class Visitor(c_ast.NodeVisitor):
  
    def __init__(self,current_parent,op):
        self.current_parent = current_parent
        self.op = op
        pass
    
    #def visit(self, node:c_ast.FileAST):
    #    #print(node)
    #    self.generic_visit(node);
   
    
    
    def visit(self, node):
        """ Visit a node.
        """

        if self._method_cache is None:
            self._method_cache = {}

        visitor = self._method_cache.get(node.__class__.__name__, None)
        if visitor is None:
            method = 'visit_' + node.__class__.__name__
            visitor = getattr(self, method, self.generic_visit)
            self._method_cache[node.__class__.__name__] = visitor

        return visitor(node)
        

    def generic_visit(self, node):
        """ Called if no explicit visitor function exists for a
            node. Implements preorder visiting of the node.
        """
        oldparent = self.current_parent
        self.current_parent = node
        op.append((get_token(oldparent),get_token(node)))
        for c in node:
            self.visit(c)
        self.current_parent = oldparent

'''

In [ ]:
#type(c_code)

pycparser.c_ast.FileAST